# Columns and Expressions

## Prerrequisites

Install Spark and Java in VM

In [ ]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark 3.5.0
!wget -q https://apache.osuosl.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

In [ ]:
ls -l # check the .tgz is there

total 391016
drwxr-xr-x 1 root root      4096 Jan 11 17:02 sample_data/
-rw-r--r-- 1 root root 400395283 Sep  9 02:10 spark-3.5.0-bin-hadoop3.tgz


In [ ]:
# unzip it
!tar xf spark-3.5.0-bin-hadoop3.tgz

In [ ]:
!pip install -q findspark

In [ ]:

!pip install py4j

# For maps
!pip install folium
!pip install plotly

Define the environment

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

Start Spark Session

---

In [ ]:
import findspark
findspark.init("spark-3.5.0-bin-hadoop3")# SPARK_HOME

from pyspark.sql import SparkSession

# create the session
spark = SparkSession \
        .builder \
        .appName("Columns and Expressions") \
        .master("local[*]") \
        .getOrCreate()

spark.version

'3.5.0'

In [ ]:
spark

In [ ]:
# For Pandas conversion optimization
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [ ]:
# Import sql functions
from pyspark.sql.functions import *

Download datasets

In [ ]:
!mkdir -p dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2324/master/dataset/cars.json -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2324/master/dataset/movies.json -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2324/master/dataset/more_cars.json -P /dataset
!ls /dataset

cars.json  more_cars.json  movies.json


Read JSON file

In [ ]:
carsDF = spark.read \
    .option("inferSchema", True) \
    .json("/dataset/cars.json")

## Examples

Select a column

In [ ]:
carsDF.select(col("Name")).show(3, False)

+-------------------------+
|Name                     |
+-------------------------+
|chevrolet chevelle malibu|
|buick skylark 320        |
|plymouth satellite       |
+-------------------------+
only showing top 3 rows



We can use various metods to refer a column

In [ ]:
# various select methods
carsDF.select(
    carsDF.Name,
    col("Acceleration"),
    "Weight_in_lbs"
).show(3)

+--------------------+------------+-------------+
|                Name|Acceleration|Weight_in_lbs|
+--------------------+------------+-------------+
|chevrolet chevell...|        12.0|         3504|
|   buick skylark 320|        11.5|         3693|
|  plymouth satellite|        11.0|         3436|
+--------------------+------------+-------------+
only showing top 3 rows



Expressions. We can use SQL like expression inside select to make operations with a column

In [ ]:
#
carsWithKgDF = carsDF.select(
    col("Name"),
    col("Weight_in_lbs"),
    (col("Weight_in_lbs")/2.2).cast("int").alias("Weight_in_kg_2"), #cast result to int
    expr("Weight_in_lbs / 1000").cast("string").alias("Weight_in_T") #cast result to str
)
carsWithKgDF.printSchema()
carsWithKgDF.show(3)

root
 |-- Name: string (nullable = true)
 |-- Weight_in_lbs: long (nullable = true)
 |-- Weight_in_kg_2: integer (nullable = true)
 |-- Weight_in_T: string (nullable = true)

+--------------------+-------------+--------------+-----------+
|                Name|Weight_in_lbs|Weight_in_kg_2|Weight_in_T|
+--------------------+-------------+--------------+-----------+
|chevrolet chevell...|         3504|          1592|      3.504|
|   buick skylark 320|         3693|          1678|      3.693|
|  plymouth satellite|         3436|          1561|      3.436|
+--------------------+-------------+--------------+-----------+
only showing top 3 rows



In [ ]:
# with expressions
carsWithSelectExprWeightsDF = carsDF.selectExpr(
    "Name",
    "Weight_in_lbs",
    "Weight_in_lbs / 2.2"
  )
carsWithSelectExprWeightsDF.show(3)

+--------------------+-------------+---------------------+
|                Name|Weight_in_lbs|(Weight_in_lbs / 2.2)|
+--------------------+-------------+---------------------+
|chevrolet chevell...|         3504|          1592.727273|
|   buick skylark 320|         3693|          1678.636364|
|  plymouth satellite|         3436|          1561.818182|
+--------------------+-------------+---------------------+
only showing top 3 rows



### DF Processing

Add a column

In [ ]:
carsWithKg3DF = carsDF.withColumn("Weight_in_kg_3", col("Weight_in_lbs") / 2.2)
carsWithKg3DF.show(3)

+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+------------------+
|Acceleration|Cylinders|Displacement|Horsepower|Miles_per_Gallon|                Name|Origin|Weight_in_lbs|      Year|    Weight_in_kg_3|
+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+------------------+
|        12.0|        8|       307.0|       130|            18.0|chevrolet chevell...|   USA|         3504|1970-01-01|1592.7272727272725|
|        11.5|        8|       350.0|       165|            15.0|   buick skylark 320|   USA|         3693|1970-01-01|1678.6363636363635|
|        11.0|        8|       318.0|       150|            18.0|  plymouth satellite|   USA|         3436|1970-01-01|1561.8181818181818|
+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+------------------+
only showing top 3 rows



Rename a column

In [ ]:
carsWithColumnRenamed = carsDF.withColumnRenamed("Weight_in_lbs", "Weight in pounds")
carsWithColumnRenamed.show(3)

+------------+---------+------------+----------+----------------+--------------------+------+----------------+----------+
|Acceleration|Cylinders|Displacement|Horsepower|Miles_per_Gallon|                Name|Origin|Weight in pounds|      Year|
+------------+---------+------------+----------+----------------+--------------------+------+----------------+----------+
|        12.0|        8|       307.0|       130|            18.0|chevrolet chevell...|   USA|            3504|1970-01-01|
|        11.5|        8|       350.0|       165|            15.0|   buick skylark 320|   USA|            3693|1970-01-01|
|        11.0|        8|       318.0|       150|            18.0|  plymouth satellite|   USA|            3436|1970-01-01|
+------------+---------+------------+----------+----------------+--------------------+------+----------------+----------+
only showing top 3 rows



In [ ]:
# careful with column names
# carsWithColumnRenamed.selectExpr("Weight in pounds")

In [ ]:
# as we hace special characters (spaces) we have to use the ``
carsWithColumnRenamed.selectExpr("`Weight in pounds`").show(3)

+----------------+
|Weight in pounds|
+----------------+
|            3504|
|            3693|
|            3436|
+----------------+
only showing top 3 rows



Remove a column

In [ ]:
carsWithColumnRenamed.printSchema()

root
 |-- Acceleration: double (nullable = true)
 |-- Cylinders: long (nullable = true)
 |-- Displacement: double (nullable = true)
 |-- Horsepower: long (nullable = true)
 |-- Miles_per_Gallon: double (nullable = true)
 |-- Name: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Weight in pounds: long (nullable = true)
 |-- Year: string (nullable = true)



In [ ]:
dropColsDF = carsWithColumnRenamed.drop("Cylinders", "Displacement")
dropColsDF.printSchema()


root
 |-- Acceleration: double (nullable = true)
 |-- Horsepower: long (nullable = true)
 |-- Miles_per_Gallon: double (nullable = true)
 |-- Name: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Weight in pounds: long (nullable = true)
 |-- Year: string (nullable = true)



Filtering

In [ ]:
nonUSCarsDF = carsDF.filter(col("Origin") != "USA")
nonUSCarsDF2 = carsDF.where(col("Origin") != "USA")
nonUSCarsDF.show(3)
print(f"{nonUSCarsDF.count()} == {nonUSCarsDF2.count()}")

+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+
|Acceleration|Cylinders|Displacement|Horsepower|Miles_per_Gallon|                Name|Origin|Weight_in_lbs|      Year|
+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+
|        17.5|        4|       133.0|       115|            NULL|citroen ds-21 pallas|Europe|         3090|1970-01-01|
|        15.0|        4|       113.0|        95|            24.0|toyota corona mar...| Japan|         2372|1970-01-01|
|        14.5|        4|        97.0|        88|            27.0|        datsun pl510| Japan|         2130|1970-01-01|
+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+
only showing top 3 rows

152 == 152


In [ ]:
# filtering with expression strings
americanCarsDF = carsDF.filter("Origin = 'USA'")
americanCarsDF.show(3)

+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+
|Acceleration|Cylinders|Displacement|Horsepower|Miles_per_Gallon|                Name|Origin|Weight_in_lbs|      Year|
+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+
|        12.0|        8|       307.0|       130|            18.0|chevrolet chevell...|   USA|         3504|1970-01-01|
|        11.5|        8|       350.0|       165|            15.0|   buick skylark 320|   USA|         3693|1970-01-01|
|        11.0|        8|       318.0|       150|            18.0|  plymouth satellite|   USA|         3436|1970-01-01|
+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+
only showing top 3 rows



Chain filters

In [ ]:
americanPowerfulCarsDF = carsDF.filter(col("Origin") == "USA").filter(col("Horsepower") > 150)
americanPowerfulCarsDF2 = carsDF.filter((col("Origin") == "USA") & (col("Horsepower") > 150))
americanPowerfulCarsDF3 = carsDF.filter("Origin = 'USA' and Horsepower > 150")
americanPowerfulCarsDF.show(3)

+------------+---------+------------+----------+----------------+-----------------+------+-------------+----------+
|Acceleration|Cylinders|Displacement|Horsepower|Miles_per_Gallon|             Name|Origin|Weight_in_lbs|      Year|
+------------+---------+------------+----------+----------------+-----------------+------+-------------+----------+
|        11.5|        8|       350.0|       165|            15.0|buick skylark 320|   USA|         3693|1970-01-01|
|        10.0|        8|       429.0|       198|            15.0| ford galaxie 500|   USA|         4341|1970-01-01|
|         9.0|        8|       454.0|       220|            14.0| chevrolet impala|   USA|         4354|1970-01-01|
+------------+---------+------------+----------+----------------+-----------------+------+-------------+----------+
only showing top 3 rows



## Exercises
1. Read the movies DF and select 2 columns of your choice
2. Create another column summing up the total profit of the movies = US_Gross + Worldwide_Gross + DVD sales. Are you obtaining nulls? How you can solve it?
3. Select all COMEDY movies with IMDB rating above 6
Use as many versions as possible

Exercise 1

In [ ]:
moviesDF = spark.read \
    .option("inferSchema", True) \
    .json("/dataset/movies.json")

moviesDF.printSchema()
moviesDF.select(col("Director"),col("Distributor")).show(3, False)

root
 |-- Creative_Type: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Distributor: string (nullable = true)
 |-- IMDB_Rating: double (nullable = true)
 |-- IMDB_Votes: long (nullable = true)
 |-- MPAA_Rating: string (nullable = true)
 |-- Major_Genre: string (nullable = true)
 |-- Production_Budget: long (nullable = true)
 |-- Release_Date: string (nullable = true)
 |-- Rotten_Tomatoes_Rating: long (nullable = true)
 |-- Running_Time_min: long (nullable = true)
 |-- Source: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- US_DVD_Sales: long (nullable = true)
 |-- US_Gross: long (nullable = true)
 |-- Worldwide_Gross: long (nullable = true)

+--------+-----------+
|Director|Distributor|
+--------+-----------+
|NULL    |Gramercy   |
|NULL    |Strand     |
|NULL    |Lionsgate  |
+--------+-----------+
only showing top 3 rows



Exercise 2

In [ ]:
moviesDF = moviesDF.withColumn(
    "Total_Profit",
    coalesce(col("US_Gross") + col("Worldwide_Gross") + col("US_DVD_sales"))
)
moviesDF.show(5, False)

+--------------------+--------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------------+------------+--------+---------------+------------+
|Creative_Type       |Director|Distributor|IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|Source             |Title                     |US_DVD_Sales|US_Gross|Worldwide_Gross|Total_Profit|
+--------------------+--------+-----------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------------+------------+--------+---------------+------------+
|NULL                |NULL    |Gramercy   |6.1        |1071      |R          |NULL       |8000000          |12-Jun-98   |NULL                  |NULL            |NULL               |The Land Girls            |NULL        

Exercise 3

In [ ]:
moviesDF.createOrReplaceTempView("movies_table")
resultDF = spark.sql("""
    SELECT *
    FROM movies_table
    WHERE Major_Genre = 'Comedy' AND IMDB_Rating > 6
""")

# Show the result DataFrame
resultDF.show(5, False)

+--------------------+----------------+-------------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------------+------------+--------+---------------+------------+
|Creative_Type       |Director        |Distributor  |IMDB_Rating|IMDB_Votes|MPAA_Rating|Major_Genre|Production_Budget|Release_Date|Rotten_Tomatoes_Rating|Running_Time_min|Source             |Title                     |US_DVD_Sales|US_Gross|Worldwide_Gross|Total_Profit|
+--------------------+----------------+-------------+-----------+----------+-----------+-----------+-----------------+------------+----------------------+----------------+-------------------+--------------------------+------------+--------+---------------+------------+
|NULL                |NULL            |Lionsgate    |6.8        |865       |NULL       |Comedy     |250000           |28-Aug-98   |NULL                  |NULL            |NULL               

In [ ]:
#ejemplo del funcionamiento de fillna

from pyspark.sql import SparkSession
from pyspark.sql.functions import lit

# Crear una sesión de Spark
spark = SparkSession.builder.appName("EjemploSpark").getOrCreate()

# Crear un DataFrame de ejemplo con valores nulos
data = [("Alice", 25, None),
        ("Bob", None, 30),
        ("Charlie", 28, 35)]

columns = ["Name", "Age", "Salary"]

df = spark.createDataFrame(data, columns)

# Mostrar el DataFrame original
print("Original DataFrame:")
df.show()

# Rellenar los valores nulos con un valor constante
df_filled = df.na.fill(0, subset=["Age", "Salary"])

# Mostrar el DataFrame después de llenar los valores nulos
print("DataFrame después de fillna:")
df_filled.show()

Original DataFrame:
+-------+----+------+
|   Name| Age|Salary|
+-------+----+------+
|  Alice|  25|  NULL|
|    Bob|NULL|    30|
|Charlie|  28|    35|
+-------+----+------+

DataFrame después de fillna:
+-------+---+------+
|   Name|Age|Salary|
+-------+---+------+
|  Alice| 25|     0|
|    Bob|  0|    30|
|Charlie| 28|    35|
+-------+---+------+

